# Load Libraries

In [1]:
import pandas as pd
from urllib.parse import urlparse
import numpy as np
import re
from tld import get_tld
from typing import Tuple, Union, Any
import pickle

In [2]:
url='https://www.youtube.com/'

# Load blacklist dataset

In [3]:
blacklist = pd.read_csv('blacklist.csv')
blacklist

,id,dateadded,url,url_status,last_online,threat,tags,urlhaus_link,reporter
0,2711161,2023-09-11 12:19:05,http://158.255.82.96:59667/Mozi.m,online,2023-09-11 13:01:06,malware_download,"elf,Mozi",https://urlhaus.abuse.ch/url/2711161/,lrz_urlhaus
1,2711160,2023-09-11 12:08:04,https://wtools.io/paste-code/bPC5,offline,NaN,malware_download,"exe,remcos",https://urlhaus.abuse.ch/url/2711160/,pmelson
2,2711159,2023-09-11 12:04:07,http://200.58.90.122:53819/Mozi.m,online,2023-09-11 12:54:32,malware_download,"elf,mirai,Mozi",https://urlhaus.abuse.ch/url/2711159/,lrz_urlhaus
3,2711158,2023-09-11 10:57:04,https://pasteio.com/raw/xJi8wKKO06j3,offline,NaN,malware_download,"exe,remcos",https://urlhaus.abuse.ch/url/2711158/,pmelson
4,2711157,2023-09-11 10:56:04,https://wtools.io/paste-code/bPAh,offline,NaN,malware_download,"dcrat,exe",https://urlhaus.abuse.ch/url/2711157/,pmelson
...,...,...,...,...,...,...,...,...,...
54413,117832,2019-02-05 20:10:03,http://azmeasurement.com/SWIFT-COPY-MT103-PAYM...,online,2023-09-11 12:49:50,malware_download,"Formbook,zip",https://urlhaus.abuse.ch/url/117832/,zbetcheckin
54414,111417,2019-01-27 14:38:06,http://t.honker.info:8/445.exe,online,2023-09-11 12:54:50,malware_download,"CoinMiner,EBDP",https://urlhaus.abuse.ch/url/111417/,anonymous
54415,101043,2019-01-02 10:53:06,http://livetrack.in/EmployeeMasterImages/qace.jpg,online,2023-09-11 13:07:57,malware_download,"arkei,Encoded,Task",https://urlhaus.abuse.ch/url/101043/,anonymous
54416,86646,2018-11-28 23:08:05,http://123.194.235.37:49320/.i,online,2023-09-11 12:52:15,malware_download,"elf,hajime",https://urlhaus.abuse.ch/url/86646/,zbetcheckin


In [4]:
if url in blacklist['url'].values:
    print('yes')
else:
    print('no')

yes


# Load Random Forest trained model file

In [5]:
with open("rf.pkl", "rb") as file:
    model = pickle.load(file)

# Feature Extraction

In [6]:
def is_url_ip_address(url: str) -> bool:
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4 with port
        
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
        '([0-9]+(?:\.[0-9]+){3}:[0-9]+)|'
        '((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)', url)  # Ipv6
    if match:
        return 1
    else:
        return 0
    

In [7]:
def process_tld(url,fix_protos):
    res = get_tld(url, as_object = True, fail_silently=False, fix_protocol=fix_protos)
    domain = res.domain    
    return domain
    
def process_url_with_tld(url,ip):
    try:
        if ip == 0:
            if str(url).startswith('http:'):
                return process_tld(url)
            else:
                return process_tld(url, fix_protos=True)
        else:
            domain = None
            return domain
    except:

        return 0

In [8]:
def get_url_path(url: str) -> Union[str, None]:
    try:
        res = get_tld(url, as_object = True, fail_silently=False, fix_protocol=True)
        if res.parsed_url.query:
            joined = res.parsed_url.path + res.parsed_url.query
            return joined
        else:
            return res.parsed_url.path
    except:
        return None
    
    
def check_domain(domain_name):
    domain_=pd.read_csv('cleaned_url.csv')
    res=1
    for x in range(len(domain_)):
        safe_domain=domain_.iloc[x].values[6]
        if domain_name==safe_domain:
            id_ =domain_.iloc[x].values[0]
            safe=domain_.iloc[id_].values[3]
            if safe==0:
                res=0
                return res
            elif safe==1:
                res=1
                return res
            else:
                pass
        else:
                pass
            
            
def alpha_count(url: str) -> int:
    alpha = 0
    for i in url:
        if i.isalpha():
            alpha += 1
    return alpha

def digit_count(url: str) -> int:
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

def count_dir_in_url_path(url_path: Union[str, None]) -> int:

    if url_path:
        n_dirs = url_path.count('/')
        return n_dirs
    else:
        return 0
    
def fld_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0


In [9]:

ip = is_url_ip_address(url)
hostname_length = len(urlparse(url).netloc)
path_length = len(urlparse(url).path)
domain_name =process_url_with_tld(url,ip)
fld_length = fld_length(url)
feature_1 = url.count('-')
feature_2 = url.count('@')
feature_3 = url.count('?')
feature_4 = url.count('%')
feature_5 = url.count('.')
feature_6 = url.count('=')
http = url.count('http')
https = url.count('https')
feature_7 = url.count('www')
digits = digit_count(url)
letters = alpha_count(url)
dri = count_dir_in_url_path(url)

output = [hostname_length, path_length, fld_length, feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, http, https, feature_7, digits, letters, dri, ip]
print(output)
features = np.array([output]) 

[19, 7, 6, 0, 0, 0, 0, 4, 0, 1, 0, 0, 15, 9, 3, 1]


# Prediction

In [12]:
import time
t0 = time.time()

result=check_domain(domain_name)
print(result)
test=''
dd=pd.read_csv('cleaned_url.csv')
if result==0:
    if url in dd['url'].values: 
        pred_test = model.predict(features)
        test=pred_test[0]
    else:
        test= 0
else:
    pred_test = model.predict(features)
    test=pred_test[0] 
training_time=time.time()-t0
print('Testing time',training_time)

None
Testing time 145.2630398273468


C:\Users\ACSASS\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [172]:
print(test)

1
